Design Strategy

![Alt text](Screenshot%20from%202022-11-12%2009-22-18.png)

#### Implementation

In [1]:
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional

class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal


    def total(self) -> Decimal:
        return self.price * self.quantity
    

class Order(NamedTuple):
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self):
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))
    
    def due(self):
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self) -> str:
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'
    

class Promotion(ABC):
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):

    def discount(self, order: Order):
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)
    

class BulkItemPromo(Promotion):
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount
    
class LargeOrderPromot(Promotion):
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [3]:
joe = Customer('John Doe', 0)
ann = Customer('Ann SMith', 1100)
cart = (LineItem('banana',4, Decimal('0.5')),
LineItem('apple',10, Decimal('1.5')),
    LineItem('watermelon',5,Decimal(5)))

Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [5]:
banana_cart = (LineItem('banana', 30, Decimal('.5')), LineItem('apple', 10, Decimal('1.5')))

In [7]:
Order(joe, banana_cart, BulkItemPromo())
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))

In [8]:
Order(joe, long_cart, LargeOrderPromot())

<Order total: 10.00 due: 9.30>

Finding Strategies in a Module

The Command Pattern

![Alt text](Screenshot%20from%202022-11-12%2015-59-35.png)